In [1]:
import pandas as pd
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [2]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": [32.806671, 61.370716, 33.729759, 34.969704, 36.116203, 39.059811, 41.597782, 39.318523, 38.897438,
                 27.766279, 33.040619, 21.094318, 44.240459, 40.349457, 39.849426, 42.011539, 38.5266, 37.66814,
                 31.169546, 44.693947, 39.063946, 42.230171, 43.326618, 45.694454, 32.741646, 38.456085, 46.921925,
                 41.12537, 38.313515, 43.452492, 40.298904, 34.840515, 42.165726, 35.630066, 47.528912, 40.388783,
                 35.565342, 44.572021, 40.590752, 41.680893, 33.856892, 44.299782, 35.747845, 31.054487, 40.150032,
                 44.045876, 37.769337, 47.400902, 38.491226, 44.268543, 42.755966],
    "longitude": [-86.79113, -152.404419, -111.431221, -92.373123, -119.681564, -105.311104, -72.755371, -75.507141,
                  -77.026817, -81.686783, -83.643074, -157.498337, -114.478828, -88.986137, -86.258278, -93.210526,
                  -96.726486, -84.670067, -91.867805, -69.381927, -76.802101, -71.530106, -84.536095, -93.900192,
                  -89.678696, -92.288368, -110.454353, -98.268082, -117.055374, -71.563896, -74.521011, -106.248482,
                  -74.948051, -79.806419, -99.784012, -82.764915, -96.928917, -122.070938, -77.209755, -71.51178,
                  -80.945007, -99.438828, -86.692345, -97.563461, -111.862434, -72.710686, -78.169968, -121.490494,
                  -80.954453, -89.616508, -107.30249],
    "start_date": "2000-01-31",
    "end_date": "2023-12-31",
    "hourly": "temperature_2m",
    "daily": ["temperature_2m_max", "temperature_2m_min"],
    "temperature_unit": "fahrenheit",
    "wind_speed_unit": "mph",
    "precipitation_unit": "inch"
}
responses = openmeteo.weather_api(url, params=params)

# Initialize lists to hold hourly and daily data
hourly_dataframes = []
daily_dataframes = []

# Loop through all responses to process data for each location
for response in responses:
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_data = {
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        ),
        "latitude": response.Latitude(),
        "longitude": response.Longitude(),
        "temperature_2m": hourly_temperature_2m
    }
    hourly_dataframes.append(pd.DataFrame(data=hourly_data))

    daily = response.Daily()
    daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
    daily_data = {
        "date": pd.date_range(
            start=pd.to_datetime(daily.Time(), unit="s", utc=True),
            end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=daily.Interval()),
            inclusive="left"
        ),
        "latitude": response.Latitude(),
        "longitude": response.Longitude(),
        "temperature_2m_max": daily_temperature_2m_max,
        "temperature_2m_min": daily_temperature_2m_min
    }
    daily_dataframes.append(pd.DataFrame(data=daily_data))

# Concatenate hourly and daily dataframes for all locations
hourly_dataframe = pd.concat(hourly_dataframes, ignore_index=True)
daily_dataframe = pd.concat(daily_dataframes, ignore_index=True)

# print(hourly_dataframe)
# print(daily_dataframe)


In [3]:
daily_dataframe

,date,latitude,longitude,temperature_2m_max,temperature_2m_min
0,2000-01-31 00:00:00+00:00,32.794376,-86.809296,42.799099,27.679100
1,2000-02-01 00:00:00+00:00,32.794376,-86.809296,51.619099,28.489100
2,2000-02-02 00:00:00+00:00,32.794376,-86.809296,49.189098,29.569099
3,2000-02-03 00:00:00+00:00,32.794376,-86.809296,53.599098,27.589100
4,2000-02-04 00:00:00+00:00,32.794376,-86.809296,52.429100,35.869099
...,...,...,...,...,...
445531,2023-12-27 00:00:00+00:00,42.776798,-107.440826,20.932699,6.802700
445532,2023-12-28 00:00:00+00:00,42.776798,-107.440826,19.042700,6.982700
445533,2023-12-29 00:00:00+00:00,42.776798,-107.440826,22.372700,12.202702
445534,2023-12-30 00:00:00+00:00,42.776798,-107.440826,25.522699,15.622700


In [4]:
# daily_dataframe.index.name = 'State'
daily_dataframe['lat'] = daily_dataframe['latitude'].astype(str).str[:2]
daily_dataframe['lon'] = daily_dataframe['longitude'].astype(str).str[:3]
daily_dataframe['concat'] = daily_dataframe['lat']+daily_dataframe['lon']
daily_dataframe

,date,latitude,longitude,temperature_2m_max,temperature_2m_min,lat,lon,concat
0,2000-01-31 00:00:00+00:00,32.794376,-86.809296,42.799099,27.679100,32,-86,32-86
1,2000-02-01 00:00:00+00:00,32.794376,-86.809296,51.619099,28.489100,32,-86,32-86
2,2000-02-02 00:00:00+00:00,32.794376,-86.809296,49.189098,29.569099,32,-86,32-86
3,2000-02-03 00:00:00+00:00,32.794376,-86.809296,53.599098,27.589100,32,-86,32-86
4,2000-02-04 00:00:00+00:00,32.794376,-86.809296,52.429100,35.869099,32,-86,32-86
...,...,...,...,...,...,...,...,...
445531,2023-12-27 00:00:00+00:00,42.776798,-107.440826,20.932699,6.802700,42,-10,42-10
445532,2023-12-28 00:00:00+00:00,42.776798,-107.440826,19.042700,6.982700,42,-10,42-10
445533,2023-12-29 00:00:00+00:00,42.776798,-107.440826,22.372700,12.202702,42,-10,42-10
445534,2023-12-30 00:00:00+00:00,42.776798,-107.440826,25.522699,15.622700,42,-10,42-10


In [5]:
state_merge = pd.read_csv("C:/Github/Group3_Project4/pete files/state weather csv/state lat lon.csv")
state_merge.head()

,State,Latitude,Longitude,reduced,reduced.1,concat,region,sub-region,country
0,Alabama,32.806671,-86.791130,32,-86,32-86,South,East South Central,US
1,Alaska,61.370716,-152.404419,61,-15,61-15,West,Pacific,US
2,Arizona,33.729759,-111.431221,33,-11,33-11,West,Mountain,US
3,Arkansas,34.969704,-92.373123,34,-92,34-92,South,West South Central,US
4,California,36.116203,-119.681564,36,-11,36-11,West,Pacific,US


In [6]:
merged_weather = pd.merge(daily_dataframe, state_merge, on = 'concat', how = 'left')
merged_weather

,date,latitude,longitude,temperature_2m_max,temperature_2m_min,lat,lon,concat,State,Latitude,Longitude,reduced,reduced.1,region,sub-region,country
0,2000-01-31 00:00:00+00:00,32.794376,-86.809296,42.799099,27.679100,32,-86,32-86,Alabama,32.806671,-86.79113,32,-86,South,East South Central,US
1,2000-02-01 00:00:00+00:00,32.794376,-86.809296,51.619099,28.489100,32,-86,32-86,Alabama,32.806671,-86.79113,32,-86,South,East South Central,US
2,2000-02-02 00:00:00+00:00,32.794376,-86.809296,49.189098,29.569099,32,-86,32-86,Alabama,32.806671,-86.79113,32,-86,South,East South Central,US
3,2000-02-03 00:00:00+00:00,32.794376,-86.809296,53.599098,27.589100,32,-86,32-86,Alabama,32.806671,-86.79113,32,-86,South,East South Central,US
4,2000-02-04 00:00:00+00:00,32.794376,-86.809296,52.429100,35.869099,32,-86,32-86,Alabama,32.806671,-86.79113,32,-86,South,East South Central,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445531,2023-12-27 00:00:00+00:00,42.776798,-107.440826,20.932699,6.802700,42,-10,42-10,Wyoming,42.755966,-107.30249,42,-10,West,Mountain,US
445532,2023-12-28 00:00:00+00:00,42.776798,-107.440826,19.042700,6.982700,42,-10,42-10,Wyoming,42.755966,-107.30249,42,-10,West,Mountain,US
445533,2023-12-29 00:00:00+00:00,42.776798,-107.440826,22.372700,12.202702,42,-10,42-10,Wyoming,42.755966,-107.30249,42,-10,West,Mountain,US
445534,2023-12-30 00:00:00+00:00,42.776798,-107.440826,25.522699,15.622700,42,-10,42-10,Wyoming,42.755966,-107.30249,42,-10,West,Mountain,US


In [7]:
weather = merged_weather.drop(columns = ['lat', 'lon', 'concat', 'Latitude', 'Longitude', 'reduced','reduced.1'])
weather.set_index('State', inplace=True)
weather

,date,latitude,longitude,temperature_2m_max,temperature_2m_min,region,sub-region,country
State,,,,,,,,
Alabama,2000-01-31 00:00:00+00:00,32.794376,-86.809296,42.799099,27.679100,South,East South Central,US
Alabama,2000-02-01 00:00:00+00:00,32.794376,-86.809296,51.619099,28.489100,South,East South Central,US
Alabama,2000-02-02 00:00:00+00:00,32.794376,-86.809296,49.189098,29.569099,South,East South Central,US
Alabama,2000-02-03 00:00:00+00:00,32.794376,-86.809296,53.599098,27.589100,South,East South Central,US
Alabama,2000-02-04 00:00:00+00:00,32.794376,-86.809296,52.429100,35.869099,South,East South Central,US
...,...,...,...,...,...,...,...,...
Wyoming,2023-12-27 00:00:00+00:00,42.776798,-107.440826,20.932699,6.802700,West,Mountain,US
Wyoming,2023-12-28 00:00:00+00:00,42.776798,-107.440826,19.042700,6.982700,West,Mountain,US
Wyoming,2023-12-29 00:00:00+00:00,42.776798,-107.440826,22.372700,12.202702,West,Mountain,US


In [8]:
weather = weather.rename(columns = {'temperature_2m_max': 'temp_max', 'temperature_2m_min': 'temp_min'})
weather

,date,latitude,longitude,temp_max,temp_min,region,sub-region,country
State,,,,,,,,
Alabama,2000-01-31 00:00:00+00:00,32.794376,-86.809296,42.799099,27.679100,South,East South Central,US
Alabama,2000-02-01 00:00:00+00:00,32.794376,-86.809296,51.619099,28.489100,South,East South Central,US
Alabama,2000-02-02 00:00:00+00:00,32.794376,-86.809296,49.189098,29.569099,South,East South Central,US
Alabama,2000-02-03 00:00:00+00:00,32.794376,-86.809296,53.599098,27.589100,South,East South Central,US
Alabama,2000-02-04 00:00:00+00:00,32.794376,-86.809296,52.429100,35.869099,South,East South Central,US
...,...,...,...,...,...,...,...,...
Wyoming,2023-12-27 00:00:00+00:00,42.776798,-107.440826,20.932699,6.802700,West,Mountain,US
Wyoming,2023-12-28 00:00:00+00:00,42.776798,-107.440826,19.042700,6.982700,West,Mountain,US
Wyoming,2023-12-29 00:00:00+00:00,42.776798,-107.440826,22.372700,12.202702,West,Mountain,US


In [9]:
weather.to_csv("state weather csv/2000-2023_weather.csv")

In [14]:
five_year_weather = weather.loc[(weather['date'] >= '2015-01-01') &  (weather['date'] <= '2020-12-31')]
five_year_weather

,date,latitude,longitude,temp_max,temp_min,region,sub-region,country
State,,,,,,,,
Alabama,2015-01-01 00:00:00+00:00,32.794376,-86.809296,53.779099,35.689098,South,East South Central,US
Alabama,2015-01-02 00:00:00+00:00,32.794376,-86.809296,52.339100,43.879097,South,East South Central,US
Alabama,2015-01-03 00:00:00+00:00,32.794376,-86.809296,67.099098,52.069099,South,East South Central,US
Alabama,2015-01-04 00:00:00+00:00,32.794376,-86.809296,66.199097,50.899101,South,East South Central,US
Alabama,2015-01-05 00:00:00+00:00,32.794376,-86.809296,49.369099,34.339100,South,East South Central,US
...,...,...,...,...,...,...,...,...
Wyoming,2020-12-27 00:00:00+00:00,42.776798,-107.440826,23.722700,8.512699,West,Mountain,US
Wyoming,2020-12-28 00:00:00+00:00,42.776798,-107.440826,17.152700,0.592701,West,Mountain,US
Wyoming,2020-12-29 00:00:00+00:00,42.776798,-107.440826,12.652700,0.772701,West,Mountain,US


In [15]:
five_year_weather.to_csv("state weather csv/5yr_weather_data.csv")

In [16]:
ten_year_weather = weather.loc[(weather['date'] >= '2010-01-01') &  (weather['date'] <= '2020-12-31')]
ten_year_weather

,date,latitude,longitude,temp_max,temp_min,region,sub-region,country
State,,,,,,,,
Alabama,2010-01-01 00:00:00+00:00,32.794376,-86.809296,49.459099,39.919098,South,East South Central,US
Alabama,2010-01-02 00:00:00+00:00,32.794376,-86.809296,41.269100,29.209101,South,East South Central,US
Alabama,2010-01-03 00:00:00+00:00,32.794376,-86.809296,37.759098,25.789101,South,East South Central,US
Alabama,2010-01-04 00:00:00+00:00,32.794376,-86.809296,32.629101,23.629099,South,East South Central,US
Alabama,2010-01-05 00:00:00+00:00,32.794376,-86.809296,34.699100,19.399099,South,East South Central,US
...,...,...,...,...,...,...,...,...
Wyoming,2020-12-27 00:00:00+00:00,42.776798,-107.440826,23.722700,8.512699,West,Mountain,US
Wyoming,2020-12-28 00:00:00+00:00,42.776798,-107.440826,17.152700,0.592701,West,Mountain,US
Wyoming,2020-12-29 00:00:00+00:00,42.776798,-107.440826,12.652700,0.772701,West,Mountain,US


In [17]:
ten_year_weather.to_csv("state weather csv/10yr_weather_data.csv")